# Database Schema Differences

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
def create_mysql_engine(host, db, user, pwd):

    maria_db_host = host
    maria_db_user = user
    maria_db_pass = pwd

    # sqlalchemy engine
    sa_cnx_str = 'mysql+pymysql://{user}:{pw}@{host}/{db}'.format(user=maria_db_user,
                                                                  pw=maria_db_pass,
                                                                  host=maria_db_host,
                                                                  db=db)
    return create_engine(sa_cnx_str, pool_pre_ping=True)


In [3]:

def create_mssql_engine(host, db, user, pwd):
    pymssql_url = 'mssql+pymssql://{u}:{p}@{h}/{d}?charset=utf8'
    return create_engine(pymssql_url.format(u=user, h=host, p=pwd, d=db), pool_pre_ping=True)

In [4]:
cnx_target = create_mssql_engine(host='ares-migration.database.windows.net',
                                     user='ares@ares-migration',
                                     pwd='ahT9sLc)3e@#',
                                     db='ares_schema_test')

In [5]:
host='data-merge-source.mariadb.database.azure.com'
user='data_admin@data-merge-source'
pwd=',a.FQx389Has_Cq~'
db='mysql'

In [6]:
cnx_source = create_mysql_engine(host='data-merge-source.mariadb.database.azure.com',
                                 user='data_admin@data-merge-source',
                                 pwd=',a.FQx389Has_Cq~',
                                 db='mysql')

In [7]:
# get databases we care about
database_filter = "Database.str.startswith('us') and not Database.str.endswith('logserver')"
dbs = pd.read_sql("show databases", con=cnx_source).query(database_filter)['Database'].tolist()
dbs

['usc01-1-ares_comcast',
 'usc01-1-ares_demo',
 'usc01-1-ares_involta',
 'usc01-1-ares_watchcom',
 'usc01-1-missionserver',
 'usw01-2-ares_aci',
 'usw01-2-ares_apprentice1',
 'usw01-2-ares_fltcyber',
 'usw01-2-ares_report',
 'usw01-2-gameserver_7',
 'usw01-2-gameserver_8',
 'usw01-2-gameserver_ang',
 'usw01-2-gameserver_cyberyankee',
 'usw01-2-gameserver_forscom',
 'usw01-2-gameserver_marsparta',
 'usw01-2-gameserver_nldmod',
 'usw01-3-ares_apprentice1',
 'usw01-3-ares_boa',
 'usw01-3-ares_cuboulder',
 'usw01-3-ares_dhf',
 'usw01-3-ares_enlink',
 'usw01-3-ares_gedigital',
 'usw01-3-ares_ibm',
 'usw01-3-ares_icisi',
 'usw01-3-ares_insyte',
 'usw01-3-ares_lcps',
 'usw01-3-ares_pro1',
 'usw01-3-ares_sd27j',
 'usw01-3-ares_test',
 'usw01-3-ares_titans',
 'usw01-3-ares_visa',
 'usw01-3-eval_apprentice',
 'usw01-3-gameserver',
 'usw01-3-gameserver_z3']

In [130]:
# get target schema
q = """
select 
TABLE_NAME,
COLUMN_NAME,
'{db}' as db 
from information_schema.columns
where TABLE_SCHEMA = '{db}'
AND COLUMN_NAME not in ('migration_date', 'source_db')
and TABLE_SCHEMA not in (
                    'usw01-2-ares_edu',
                    'usw01-2-ares_pro1',
                  'usw01-2-gameserver',

                  'usw01-2-gameserver_9'
  )
"""
df_reference = pd.read_sql(q.format(db='stage'), con=cnx_target)

In [143]:
# get list of target tables
df_tables = list()

for db in dbs:
    
    cnx = create_mysql_engine(host=host,
                                 user=user,
                                 pwd=pwd,
                                 db=db)
    df = pd.read_sql(q.format(db=db), con=cnx)
    print(db)
    print(df.shape)
    if len(df):
        df_tables.append(df)


usc01-1-ares_comcast
(581, 3)
usc01-1-ares_demo
(581, 3)
usc01-1-ares_involta
(581, 3)
usc01-1-ares_watchcom
(581, 3)
usc01-1-missionserver
(0, 3)
usw01-2-ares_aci
(584, 3)
usw01-2-ares_apprentice1
(352, 3)
usw01-2-ares_fltcyber
(581, 3)
usw01-2-ares_report
(541, 3)
usw01-2-gameserver_7
(609, 3)
usw01-2-gameserver_8
(609, 3)
usw01-2-gameserver_ang
(582, 3)
usw01-2-gameserver_cyberyankee
(584, 3)
usw01-2-gameserver_forscom
(581, 3)
usw01-2-gameserver_marsparta
(581, 3)
usw01-2-gameserver_nldmod
(581, 3)
usw01-3-ares_apprentice1
(584, 3)
usw01-3-ares_boa
(581, 3)
usw01-3-ares_cuboulder
(581, 3)
usw01-3-ares_dhf
(584, 3)
usw01-3-ares_enlink
(581, 3)
usw01-3-ares_gedigital
(581, 3)
usw01-3-ares_ibm
(584, 3)
usw01-3-ares_icisi
(581, 3)
usw01-3-ares_insyte
(581, 3)
usw01-3-ares_lcps
(584, 3)
usw01-3-ares_pro1
(584, 3)
usw01-3-ares_sd27j
(581, 3)
usw01-3-ares_test
(609, 3)
usw01-3-ares_titans
(582, 3)
usw01-3-ares_visa
(579, 3)
usw01-3-eval_apprentice
(584, 3)
usw01-3-gameserver
(609, 3)
usw0

In [144]:
df_schema = pd.concat(df_tables)

In [145]:
df_schema['db'].unique()

array(['usc01-1-ares_comcast', 'usc01-1-ares_demo',
       'usc01-1-ares_involta', 'usc01-1-ares_watchcom',
       'usw01-2-ares_aci', 'usw01-2-ares_apprentice1',
       'usw01-2-ares_fltcyber', 'usw01-2-ares_report',
       'usw01-2-gameserver_7', 'usw01-2-gameserver_8',
       'usw01-2-gameserver_ang', 'usw01-2-gameserver_cyberyankee',
       'usw01-2-gameserver_forscom', 'usw01-2-gameserver_marsparta',
       'usw01-2-gameserver_nldmod', 'usw01-3-ares_apprentice1',
       'usw01-3-ares_boa', 'usw01-3-ares_cuboulder', 'usw01-3-ares_dhf',
       'usw01-3-ares_enlink', 'usw01-3-ares_gedigital',
       'usw01-3-ares_ibm', 'usw01-3-ares_icisi', 'usw01-3-ares_insyte',
       'usw01-3-ares_lcps', 'usw01-3-ares_pro1', 'usw01-3-ares_sd27j',
       'usw01-3-ares_test', 'usw01-3-ares_titans', 'usw01-3-ares_visa',
       'usw01-3-eval_apprentice', 'usw01-3-gameserver',
       'usw01-3-gameserver_z3'], dtype=object)

In [146]:
foo.unique()

array(['api_scheduledevent', 'api_missionfile', 'api_sitesettings',
       'api_missionobjective', 'api_chatroom_subscribed_sessions',
       'api_comment', 'api_skill', 'api_missionobjectivecoreskill',
       'api_skilltype', 'api_missionsession', 'api_coreskill',
       'api_coreskillaward', 'api_staticasset', 'api_missionsessionfile',
       'api_invitation', 'api_trainersession',
       'api_missionsessionsettings', 'api_learningpath', 'api_workrole',
       'api_objectivehint', 'api_learningpathoption', 'auth_group',
       'api_organization', 'api_learningpathoption_locked_by',
       'auth_group_permissions', 'api_organizationsettings',
       'api_learningpathstep', 'auth_permission', 'api_organizationuser',
       'auth_user', 'api_mapmarker', 'api_playerprofile',
       'auth_user_groups', 'auth_user_user_permissions',
       'api_mediacentercategory', 'api_playersession',
       'defender_accessattempt', 'api_mediacenterquery',
       'django_admin_log', 'api_preference', 'd

In [157]:
diffs = list()
for df in df_tables:
    source_tables = set(df.TABLE_NAME.unique().tolist())
    refernce_tables = set(df_reference.TABLE_NAME.unique().tolist())
    
    df['join_flg'] = 1
    df_reference['join_flg'] = 1
    df_ref = df_reference.copy()
    df_ref['db'] = df.db.unique()[0]
    table_diffs = source_tables.symmetric_difference(refernce_tables)
    
    if list(table_diffs):
        print(df.db.unique())
        print(table_diffs)
        diffs.append(df_ref.merge(df, how='outer', on=['db','TABLE_NAME', 'COLUMN_NAME']).query("join_flg_x != join_flg_x or  join_flg_y != join_flg_y").query("TABLE_NAME in @table_diffs"))

df_diffs = pd.concat(diffs)

['usc01-1-ares_comcast']
{'api_oauthapplication', 'oauth2_provider_accesstoken', 'oauth2_provider_grant', 'oauth2_provider_refreshtoken', 'api_oauthapplication_groups'}
['usc01-1-ares_demo']
{'api_oauthapplication', 'oauth2_provider_accesstoken', 'oauth2_provider_grant', 'oauth2_provider_refreshtoken', 'api_oauthapplication_groups'}
['usc01-1-ares_involta']
{'api_oauthapplication', 'oauth2_provider_accesstoken', 'oauth2_provider_grant', 'oauth2_provider_refreshtoken', 'api_oauthapplication_groups'}
['usc01-1-ares_watchcom']
{'api_oauthapplication', 'oauth2_provider_accesstoken', 'oauth2_provider_grant', 'oauth2_provider_refreshtoken', 'api_oauthapplication_groups'}
['usw01-2-ares_aci']
{'api_oauthapplication', 'oauth2_provider_accesstoken', 'oauth2_provider_grant', 'oauth2_provider_refreshtoken', 'api_cache', 'api_oauthapplication_groups'}
['usw01-2-ares_apprentice1']
{'api_mediacenterquery', 'api_missionfile', 'api_sitesettings', 'api_minigamesessiontoken', 'api_quizquestion', 'api_me

In [158]:
df_diffs.head()

,TABLE_NAME,COLUMN_NAME,db,join_flg_x,join_flg_y
541,api_oauthapplication,id,usc01-1-ares_comcast,NaN,1.0
542,api_oauthapplication,client_id,usc01-1-ares_comcast,NaN,1.0
543,api_oauthapplication,redirect_uris,usc01-1-ares_comcast,NaN,1.0
544,api_oauthapplication,client_type,usc01-1-ares_comcast,NaN,1.0
545,api_oauthapplication,authorization_grant_type,usc01-1-ares_comcast,NaN,1.0


### Diffs that only exist in the source

In [170]:
df_diffs.query("join_flg_y ==1").groupby('db')['TABLE_NAME'].nunique().sort_values(ascending=False)

db
usw01-3-gameserver                11
usw01-3-ares_test                 11
usw01-2-gameserver_8              11
usw01-2-gameserver_7              11
usw01-3-gameserver_z3              6
usw01-3-ares_ibm                   6
usw01-2-gameserver_ang             6
usw01-3-ares_apprentice1           6
usw01-3-ares_dhf                   6
usw01-2-ares_aci                   6
usw01-3-ares_lcps                  6
usw01-3-ares_pro1                  6
usw01-3-ares_titans                6
usw01-3-eval_apprentice            6
usw01-2-gameserver_cyberyankee     6
usw01-2-gameserver_forscom         5
usc01-1-ares_watchcom              5
usw01-2-ares_fltcyber              5
usc01-1-ares_involta               5
usc01-1-ares_demo                  5
usw01-3-ares_boa                   5
usw01-2-gameserver_marsparta       5
usw01-2-gameserver_nldmod          5
usw01-3-ares_cuboulder             5
usw01-3-ares_enlink                5
usw01-3-ares_gedigital             5
usw01-3-ares_icisi                 

In [163]:
df_diffs.query("join_flg_y ==1")[['TABLE_NAME']].drop_duplicates()

,TABLE_NAME
541,api_oauthapplication
552,api_oauthapplication_groups
555,oauth2_provider_accesstoken
564,oauth2_provider_grant
573,oauth2_provider_refreshtoken
541,api_cache
554,api_watsonconversationexchange
584,social_auth_association
591,social_auth_code
595,social_auth_nonce


In [179]:
for x, y in df_diffs.query("join_flg_y ==1").groupby('TABLE_NAME'):
    print(x, y.COLUMN_NAME.unique(), '\n')



api_cache ['cache_key' 'value' 'expires'] 

api_oauthapplication ['id' 'client_id' 'redirect_uris' 'client_type' 'authorization_grant_type'
 'client_secret' 'name' 'skip_authorization' 'created' 'updated' 'user_id'] 

api_oauthapplication_groups ['id' 'oauthapplication_id' 'group_id'] 

api_watsonconversationexchange ['id' 'timestamp' 'workspace_id' 'input' 'output_text' 'context'
 'sent_context' 'intents' 'entities' 'request_duration'
 'missionsession_id' 'user_id'] 

oauth2_provider_accesstoken ['id' 'token' 'expires' 'scope' 'application_id' 'user_id' 'created'
 'updated' 'source_refresh_token_id'] 

oauth2_provider_grant ['id' 'code' 'expires' 'redirect_uri' 'scope' 'application_id' 'user_id'
 'created' 'updated'] 

oauth2_provider_refreshtoken ['id' 'token' 'access_token_id' 'application_id' 'user_id' 'created'
 'updated' 'revoked'] 

social_auth_association ['id' 'server_url' 'handle' 'secret' 'issued' 'lifetime' 'assoc_type'] 

social_auth_code ['id' 'email' 'code' 'verified'] 


In [171]:
df_diffs.query("join_flg_y ==1")[['TABLE_NAME', 'COLUMN_NAME']].drop_duplicates()

,TABLE_NAME,COLUMN_NAME
541,api_oauthapplication,id
542,api_oauthapplication,client_id
543,api_oauthapplication,redirect_uris
544,api_oauthapplication,client_type
545,api_oauthapplication,authorization_grant_type
546,api_oauthapplication,client_secret
547,api_oauthapplication,name
548,api_oauthapplication,skip_authorization
549,api_oauthapplication,created
550,api_oauthapplication,updated


### Diffs missing in source

In [161]:
df_diffs.query("join_flg_x == 1")[['db','TABLE_NAME']].drop_duplicates()

,db,TABLE_NAME
33,usw01-2-ares_apprentice1,api_mapmarker
80,usw01-2-ares_apprentice1,api_mediacentercategory
93,usw01-2-ares_apprentice1,api_mediacenterquery
99,usw01-2-ares_apprentice1,api_mediacenterresource
108,usw01-2-ares_apprentice1,defender_accessattempt
124,usw01-2-ares_apprentice1,api_mediacenterresource_core_skills
135,usw01-2-ares_apprentice1,api_questionfollowup
142,usw01-2-ares_apprentice1,api_mediacentertab
149,usw01-2-ares_apprentice1,api_quiz
166,usw01-2-ares_apprentice1,api_quiz_questions


In [156]:
df_diffs.query("join_flg_x == 1")

,TABLE_NAME,COLUMN_NAME,db_x,join_flg_x,db_y,join_flg_y
33,api_mapmarker,archived,stage,1.0,NaN,NaN
34,api_mapmarker,asset_id,stage,1.0,NaN,NaN
35,api_mapmarker,catalog_id,stage,1.0,NaN,NaN
36,api_mapmarker,coin,stage,1.0,NaN,NaN
37,api_mapmarker,complexity,stage,1.0,NaN,NaN
38,api_mapmarker,description,stage,1.0,NaN,NaN
39,api_mapmarker,id,stage,1.0,NaN,NaN
40,api_mapmarker,latitude,stage,1.0,NaN,NaN
41,api_mapmarker,location,stage,1.0,NaN,NaN
42,api_mapmarker,longitude,stage,1.0,NaN,NaN


# Natural Key Search

In [180]:
data_fixtures = ['api_badge',
                     'api_badgerequirement',
                     'api_catalogquizquestion',
                     'api_coreskill',
                     'api_learningpath',
                     'api_learningpathoption',
                     'api_learningpathoption_locked_by',
                     'api_learningpathstep',
                     'api_mapmarker',
                     'api_mediacentercategory',
                     'api_mediacenterresource',
                     'api_mediacenterresource_core_skills',
                     'api_mediacentertab',
                     'api_minigame',
                     'api_minigamedefaultsetting',
                     'api_minigamefield',
                     'api_minigamesection',
                     'api_mission',
                     'api_missioncatalogentry',
                     'api_missionfile',
                     'api_questionanswerpair',
                     'api_questionfollowup',
                     'api_quizanswer',
                     'api_quizquestion',
                     'api_quizquestion_minigame_sections',
                     'api_quizquestion_skills',
                     'api_rank',
                     'api_sitesettings',
                     'api_skilltype',
                     'api_staticasset',
                     'api_workrole',
                     'auth_permission',
                     'defender_accessattempt',
                     'django_content_type',
                     'django_migrations',
                     'django_session']

In [188]:
df_schema.query("TABLE_NAME == 'api_badge'")['db'].unique()

array(['usc01-1-ares_comcast', 'usc01-1-ares_demo',
       'usc01-1-ares_involta', 'usc01-1-ares_watchcom',
       'usw01-2-ares_aci', 'usw01-2-ares_apprentice1',
       'usw01-2-ares_fltcyber', 'usw01-2-ares_report',
       'usw01-2-gameserver_7', 'usw01-2-gameserver_8',
       'usw01-2-gameserver_ang', 'usw01-2-gameserver_cyberyankee',
       'usw01-2-gameserver_forscom', 'usw01-2-gameserver_marsparta',
       'usw01-2-gameserver_nldmod', 'usw01-3-ares_apprentice1',
       'usw01-3-ares_boa', 'usw01-3-ares_cuboulder', 'usw01-3-ares_dhf',
       'usw01-3-ares_enlink', 'usw01-3-ares_gedigital',
       'usw01-3-ares_ibm', 'usw01-3-ares_icisi', 'usw01-3-ares_insyte',
       'usw01-3-ares_lcps', 'usw01-3-ares_pro1', 'usw01-3-ares_sd27j',
       'usw01-3-ares_test', 'usw01-3-ares_titans', 'usw01-3-ares_visa',
       'usw01-3-eval_apprentice', 'usw01-3-gameserver',
       'usw01-3-gameserver_z3'], dtype=object)

In [208]:
fixture_dict = dict()
for fixture in data_fixtures:
    
    affected_schemas = df_schema.query(f"TABLE_NAME == '{fixture}'")['db'].unique().tolist()
    
    inner_list = list()
    for schema in affected_schemas:
        cnx = create_mysql_engine(host=host,
                                 user=user,
                                 pwd=pwd,
                                 db=schema)
        df_inner = pd.read_sql_table(fixture, con=cnx)
        df_inner['db'] = schema
        inner_list.append(df_inner)
    fixture_dict[fixture] = pd.concat(inner_list)
    

/Users/jremigio/anaconda3/envs/ares-migration/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [221]:
fixture_dict[k].groupby('db').apply(lambda x: x.apply(lambda y: y.is_unique)[:-1]).all()

session_key      True
session_data    False
expire_date      True
dtype: bool

In [257]:
# check for unique columns for each database
for k,df in fixture_dict.items():
    print(k)
    print(fixture_dict[k].groupby('db').apply(lambda x: x.apply(lambda y: y.is_unique)).all().sort_values(ascending=False), '\n')


api_badge
description     True
label           True
id              True
db             False
dtype: bool 

api_badgerequirement
core_skill_id     True
id                True
db               False
badge_id         False
points           False
skill_source     False
dtype: bool 

api_catalogquizquestion
id              True
db             False
question_id    False
catalog_id     False
dtype: bool 

api_coreskill
label           True
db             False
description    False
dtype: bool 

api_learningpath
workrole_id     True
name            True
id              True
uid            False
db             False
dtype: bool 

api_learningpathoption
id                      True
uid                    False
pass_percentage        False
option_type            False
object_id              False
name                   False
learningpathstep_id    False
hints_enabled          False
db                     False
content_type_id        False
archived               False
dtype: bool 

api_learningpa

In [248]:
fixture_dict['api_learningpath'].groupby(['db','uid']).count().sort_values('id',ascending=False)

id  name  \
db                             uid                                          
usc01-1-ares_comcast           16d1df5bd85d427fb5c15adafa572df8   1     1   
usw01-3-ares_insyte            6d8e3649251149c7a14187ec9d4431c0   1     1   
usw01-3-ares_ibm               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_icisi             16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431c0   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_insyte            16d1df5bd85d427fb5c15adafa572df8   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
usw01-3-ares_ibm               8d88b0ed3b224792862f0ff36d6f0e32   1     1   
usw01-3-ares_insyte            a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_lcps              16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431c0   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_ibm               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
usc01-1-ares_comcast           6d8e3649251149c7a14187ec9d4431c0   1     1   
usw01-3-ares_enlink            740be6e876ea47268c244428139514f4   1     1   
usw01-3-ares_dhf               16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431c0   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_enlink            16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431c0   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
...                                                              ..   ...   
usw01-3-ares_apprentice1       6d8e3649251149c7a14187ec9d4431c0   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
usw01-2-gameserver_ang         16d1df5bd85d427fb5c15adafa572df8   1     1   
usw01-3-ares_apprentice1       d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_boa               16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431c0   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-3-ares_cuboulder         16d1df5bd85d427fb5c15adafa572df8   1     1   
usw01-2-gameserver_marsparta   87b7da3944324418b17c05e292c53c2a   1     1   
                               7f671823cdf045d0bb2a13fa8053c4cc   1     1   
                               3d4e8d3454cb487e8a73af475c1fc8d5   1     1   
                               3789a222f2274c32910fa6b45799da95   1     1   
usw01-2-gameserver_ang         6d8e3649251149c7a14187ec9d4431c0   1     1   
                               740be6e876ea47268c244428139514f4   1     1   
                               a75d9269fd004bd4b351d9f84e5d8b2c   1     1   
                               d39325ed1f9a40fbb949b665ab6eb40c   1     1   
usw01-2-gameserver_cyberyankee 16d1df5bd85d427fb5c15adafa572df8   1     1   
                               6d8e3649251149c7a14187ec9d4431

In [256]:
fixture_dict['api_learningpathstep'].groupby('db').apply(lambda x: x.apply(lambda y: y.is_unique)).all().sort_values(ascending=False)

id                  True
uid                False
name               False
learningpath_id    False
index              False
db                 False
condition          False
dtype: bool

In [255]:
fixture_dict['api_learningpathstep'].groupby(['db', 'uid']).count().query("id > 1")

,,condition,id,index,learningpath_id,name
db,uid,,,,,
usw01-2-gameserver_7,bc11bb987b4740a68f8316bb2a7b8056,3,3,3,3,3
usw01-2-gameserver_ang,a1d7dc42be8646e89b565b22a535fc2e,3,3,3,3,3
usw01-3-ares_titans,0f9118d5557540ffa11aab88d5feaa9f,3,3,3,3,3
